In [4]:
#!pip install estnltk==1.7.3
#nltk.download('punkt_tab')
#!pip install estnltk_neural
#!pip install sentencepiece

In [1]:
import estnltk
import os
import time

from estnltk import Text
from estnltk.converters import text_to_json, json_to_text

from estnltk import download, get_resource_paths
download("estbertner")
download("estbertner_v2")
from estnltk_neural.taggers import EstBERTNERTagger
from estnltk.taggers import TimexTagger
from estnltk.taggers import AddressPartTagger, AddressGrammarTagger

from data_preprocessing import TokenizationPreprocessorFixed
from bert_ner_tagger import BertNERTagger

from nervaluate import Evaluator

from estnltk_core.layer.span_operations import conflict
import pandas as pd

Resource 'estbertner_from_tartunlp_hf_2022-05-06' has already been downloaded.
Resource 'estbertner_v2_from_tartunlp_hf_2022-12-12' has already been downloaded.


In [13]:
InputFolder = 'data/Andmed1'
OutputFolder = 'data/output'

In [46]:
neural_ner = EstBERTNERTagger()
model_location = get_resource_paths("estbertner_v2", only_latest=True)
neural_ner2 = EstBERTNERTagger(model_location=model_location, output_layer='estbertner2')
hf_model_id = 'tartuNLP/est-roberta-hist-ner'
bert_ner_tagger = BertNERTagger(bert_tokenizer_location=hf_model_id, 
                                bert_ner_location=hf_model_id, output_layer='bert_ner',
                                token_level=False, do_lower_case=False, use_fast=False)
timex_tagger = TimexTagger()
part_tagger = AddressPartTagger()
grammar_tagger = AddressGrammarTagger()

Some weights of the model checkpoint at C:\Users\Norman\anaconda3\Lib\site-packages\estnltk\estnltk_resources\estbert\ner_model_hf_tartunlp_2022-05-06\ were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda
Device set to use cuda


In [48]:
start_time = time.time()
for name in os.listdir(InputFolder):
    print(name)
    with open(InputFolder + "/" + name, "r", encoding="utf-8") as file:
        content = file.read()
    text_import = json_to_text(json_text=content)

    #text_import.meta['document_creation_time'] = text_import.meta["meeting_dates"][0]
    
    #estbertner
    neural_ner.tag(text_import)

    #estbertner_v2
    neural_ner2.tag(text_import)

    #bert_ner
    bert_ner_tagger.tag(text_import)
    
    #timex
    #timex_tagger.tag(text_import)

    #AddressTagger
    #part_tagger.tag(text_import)
    #grammar_tagger.tag(text_import)
    
    text_to_json(text_import, file=(OutputFolder + "/" + name))

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds")

Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


1922-04-24_manual_annotated.json


Token indices sequence length is longer than the specified maximum sequence length for this model (538 > 512). Running this sequence through the model will result in indexing errors
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


1927-03-28_manual_annotated.json
1932-01-25_manual_annotated.json
1934-10-15_manual_annotated.json
1935-09-30_manual_annotated.json
1936-09-07_manual_annotated.json
1941-01-03_manual_annotated.json
Elapsed time: 31.8461 seconds


In [52]:
with open(OutputFolder + "/" + "1922-04-24_manual_annotated.json", "r", encoding="utf-8") as file:
    content = file.read()

text_import = json_to_text(json_text=content)
#preprocessor.preprocess(text_import)
text_import.layers

{'bert_ner',
 'estbertner',
 'estbertner2',
 'ne_gold_a',
 'sentences',
 'textline_info',
 'words'}

In [ ]:
{'address_parts',
 'addresses',
 'bert_ner',
 'compound_tokens',
 'estbertner',
 'estbertner2',
 'manual_named_entities',
 'sentences',
 'textline_info',
 'timexes',
 'tokens',
 'words'}

In [5]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.get_device_name(0))  # Name of the first GPU

True
1
NVIDIA GeForce RTX 3080


In [15]:
tags_manual = set()
tags_estbertner = set()
tags_estbertner2 = set()
tags_bertner = set()
tags_addresses = set()
tags_timexes = set()


for name in os.listdir(OutputFolder):
    #print(name)
    with open(OutputFolder + "/" + name, "r", encoding="utf-8") as file:
        content = file.read()
    text_import = json_to_text(json_text=content)

    for i in text_import.manual_named_entities:
    #for i in text_import.ne_gold_a:
        tags_manual.add(i.tag[0])
        #tags_manual.add(i.tag)

    for i in text_import.estbertner:
        tags_estbertner.add(i.nertag)

    for i in text_import.estbertner2:
        tags_estbertner2.add(i.nertag)

    for i in text_import.bert_ner:
        tags_bertner.add(i.nertag[0])

    #for i in text_import.timexes:
     #   tags_timexes.add(i.type)

    #for i in text_import.addresses:
     #   tags_addresses.add(i.grammar_symbol[0])


In [16]:
print(tags_manual)
print(tags_estbertner)
print(tags_estbertner2)
print(tags_bertner)
print(tags_addresses)
print(tags_timexes)

{'Address', 'party', 'Event', 'date', 'organization', 'school', 'unclear', 'person', 'work', 'place'}
{'ORG', 'PER', 'LOC'}
{'LOC', 'PER', 'GPE', 'PERCENT', 'PROD', 'MONEY', 'TITLE', 'TIME', 'DATE', 'EVENT', 'ORG'}
{'LOC', 'LOC_ORG', 'PER', 'MISC', 'ORG'}
set()
set()


In [20]:
def naited(layer, tag):
    for name in os.listdir(OutputFolder):
        with open(OutputFolder + "/" + name, "r", encoding="utf-8") as file:
            content = file.read()
        text_import = json_to_text(json_text=content)

        if layer == 'manual':
            #for i in text_import.manual_named_entities:
            for i in text_import.ne_gold_a:
                #if i.tag[0] == tag:
                if i.tag == tag:
                    print(i)
        elif layer == 'estbertner' or layer == 'estbertner2':
            for i in text_import[layer]:
                if i.nertag == tag:
                    print(i)
        elif layer == 'bertner':
            for i in text_import.bert_ner:
                if i.nertag[0] == tag:
                    print(i)            
        elif layer == 'timexes':
            for i in text_import.timexes:
                if i.type == tag:
                    print(i)
        elif layer == 'addresses':
            for i in text_import.addresses:
                if i.grammar_symbol[0] == tag:
                    print(i)

In [22]:
naited('manual', 'MONEY')

Span('I-III r', [{'tag': 'MONEY'}])
Span('- 200 m.', [{'tag': 'MONEY'}])
Span('IV-VI r', [{'tag': 'MONEY'}])
Span('-150 m.', [{'tag': 'MONEY'}])
Span('VII-X r', [{'tag': 'MONEY'}])
Span('-125 m.', [{'tag': 'MONEY'}])
Span('XI-XIII\nr', [{'tag': 'MONEY'}])
Span('-100 mk.', [{'tag': 'MONEY'}])
Span('XIV-XV r.', [{'tag': 'MONEY'}])
Span('- 75 mk.', [{'tag': 'MONEY'}])
Span('XVI-XVII r', [{'tag': 'MONEY'}])
Span('- 50 mk.', [{'tag': 'MONEY'}])
Span('XVIII-XIX r', [{'tag': 'MONEY'}])
Span('-25 m', [{'tag': 'MONEY'}])
Span('I r', [{'tag': 'MONEY'}])
Span('-150 m.', [{'tag': 'MONEY'}])
Span('II r', [{'tag': 'MONEY'}])
Span('-100 m.', [{'tag': 'MONEY'}])
Span('III r', [{'tag': 'MONEY'}])
Span('-75 mk.', [{'tag': 'MONEY'}])
Span('IVr', [{'tag': 'MONEY'}])
Span('-50 mk.', [{'tag': 'MONEY'}])
Span('I r', [{'tag': 'MONEY'}])
Span('-200 mk.', [{'tag': 'MONEY'}])
Span('II ja III r', [{'tag': 'MONEY'}])
Span('-150 mk', [{'tag': 'MONEY'}])
Span('3,700,000 marga', [{'tag': 'MONEY'}])
Span('1.050.000 m.

In [24]:
naited('estbertner2', 'MONEY')

EnvelopingSpan(['15.000.000', 'marka'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['15.000.000'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['15.000.000'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['45.000.000', 'marka'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['14,000', 'marka'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['7,500', 'marka'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['7,000', 'marka'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['1000', 'marka'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['1000', 'marka'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['11.000'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['150.000', ','], [{'nertag': 'MONEY'}])
EnvelopingSpan(['20.000.000', 'senti'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['3500'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['25%'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['2000'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['1000.'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['30.'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['35'], [{'nertag': 'MONEY'}])
EnvelopingSpan(['6,50']

In [54]:
def transform_for_stats(layer):
    manual = {
        'organization': 'ORG',
        'person': 'PER',
        'party': 'ORG',
        'school': 'ORG',
        'place': 'LOC',
        'work': 'PER',
        'Address': 'LOC',
        'date': 'DATE',
        'Event': 'EVENT'
    }

    manual2 = {
        "PER": "PER",
        "LOC": "LOC",
        "ORG": "ORG",
        "ORG_POL": "ORG",
        "ORG_GPE": "ORG",
        "EVENT": "EVENT",
        "MONEY": "MONEY"
    }

    bertner = {
        "LOC": "LOC",
        "ORG": "ORG",
        "PER": "PER",
        "LOC_ORG": "ORG"
    }
    
    temp = [[] for _ in range(len(os.listdir(OutputFolder)))]
    for nr, name in enumerate(os.listdir(OutputFolder)):
        with open(OutputFolder + "/" + name, "r", encoding="utf-8") as file:
            content = file.read()
        text_import = json_to_text(json_text=content)

        if layer == 'manual':
            #for i in text_import.manual_named_entities:
            for i in text_import.ne_gold_a:
                #if i.tag[0] in list(manual.keys()):
                if i.tag in list(manual2.keys()):
                    #temp[nr].append({"label": manual[i.tag[0]], "start": i.start, "end": i.end})
                    temp[nr].append({"label": manual2[i.tag], "start": i.start, "end": i.end})
        elif layer == 'estbertner':
            for i in text_import.estbertner:
                temp[nr].append({"label": i.nertag, "start": i.start, "end": i.end})
        elif layer == 'estbertner2':
            for i in text_import.estbertner2:
                if i.nertag not in ['GPE', 'TITLE', 'PERCENT', 'PROD']:
                    temp[nr].append({"label": i.nertag, "start": i.start, "end": i.end})
        elif layer == 'bertner':
            for i in text_import.bert_ner:
                if i.nertag[0] not in ['MISC']:
                    temp[nr].append({"label": bertner[i.nertag[0]], "start": i.start, "end": i.end})
    return temp

In [56]:
true = transform_for_stats('manual')
estbertner = transform_for_stats('estbertner')
estbertner2 = transform_for_stats('estbertner2')
bert_ner = transform_for_stats('bertner')

In [57]:
estbertner_eval = Evaluator(true, estbertner, tags=['LOC', 'ORG', 'PER'])
estbertner2_eval = Evaluator(true, estbertner2, tags=['LOC', 'ORG', 'PER', 'EVENT', 'DATE', 'MONEY'])
bert_ner_eval = Evaluator(true, bert_ner, tags=['LOC', 'ORG', 'PER'])

In [58]:
results1, results_per_tag1, result_indices1, result_indices_by_tag1 = estbertner_eval.evaluate()
results2, results_per_tag2, result_indices2, result_indices_by_tag2 = estbertner2_eval.evaluate()
results3, results_per_tag3, result_indices3, result_indices_by_tag3 = bert_ner_eval.evaluate()

In [33]:
results1['strict']

{'correct': 12010,
 'incorrect': 9813,
 'partial': 0,
 'missed': 8197,
 'spurious': 7310,
 'possible': 30020,
 'actual': 29133,
 'precision': 0.41224727971715924,
 'recall': 0.4000666222518321,
 'f1': 0.40606562642638583}

In [35]:
results2['strict']

{'correct': 2664,
 'incorrect': 2811,
 'partial': 0,
 'missed': 26020,
 'spurious': 3630,
 'possible': 31495,
 'actual': 9105,
 'precision': 0.2925864909390445,
 'recall': 0.08458485473884744,
 'f1': 0.13123152709359606}

In [37]:
results3['strict']

{'correct': 10936,
 'incorrect': 9743,
 'partial': 0,
 'missed': 9347,
 'spurious': 8102,
 'possible': 30026,
 'actual': 28781,
 'precision': 0.3799728987873945,
 'recall': 0.36421767801238925,
 'f1': 0.3719285119118472}

In [78]:
manual = {
    'organization': 'ORG',
    'person': 'PER',
    'place': 'LOC',
    'date' : 'DATE',
    'Event' : 'EVENT'
}

temp = ['LOC', 'ORG', 'PER', 'EVENT']

true = [[] for _ in range(len(os.listdir(OutputFolder)))]
estbertner = [[] for _ in range(len(os.listdir(OutputFolder)))]
estbertner2 = [[] for _ in range(len(os.listdir(OutputFolder)))]
bertner = [[] for _ in range(len(os.listdir(OutputFolder)))]

for nr, name in enumerate(os.listdir(OutputFolder)):
    with open(OutputFolder + "/" + name, "r", encoding="utf-8") as file:
        content = file.read()
    text_import = json_to_text(json_text=content)

    #for i in text_import.manual_named_entities:
    for i in text_import.ne_gold_a:
        #if i.tag[0] in list(manual.keys()):
        if i.tag in temp:
            #true[nr].append({"label": manual[i.tag[0]], "start": i.start, "end": i.end})
            true[nr].append({"label": i.tag, "start": i.start, "end": i.end})
            
    for i in text_import.estbertner:
        estbertner[nr].append({"label": i.nertag, "start": i.start, "end": i.end})

    for i in text_import.estbertner2:
        if i.nertag in list(manual.values()):
            estbertner2[nr].append({"label": i.nertag, "start": i.start, "end": i.end})

    for i in text_import.bert_ner:
        if i.nertag[0] in list(manual.values()):
            bertner[nr].append({"label": i.nertag[0], "start": i.start, "end": i.end})

In [80]:
estbertner_eval = Evaluator(true, estbertner, tags=['LOC', 'ORG', 'PER'])
estbertner2_eval = Evaluator(true, estbertner2, tags=['LOC', 'ORG', 'PER', 'EVENT', 'DATE'])
bert_ner_eval = Evaluator(true, bertner, tags=['LOC', 'ORG', 'PER'])

In [82]:
results1, results_per_tag1, result_indices1, result_indices_by_tag1 = estbertner_eval.evaluate()
results2, results_per_tag2, result_indices2, result_indices_by_tag2 = estbertner2_eval.evaluate()
results3, results_per_tag3, result_indices3, result_indices_by_tag3 = bert_ner_eval.evaluate()

In [33]:
#results1['strict']
results_per_tag1

{'LOC': {'ent_type': {'correct': 2603,
   'incorrect': 767,
   'partial': 0,
   'missed': 1705,
   'spurious': 1590,
   'possible': 5075,
   'actual': 4960,
   'precision': 0.5247983870967742,
   'recall': 0.5129064039408867,
   'f1': 0.518784255107125},
  'partial': {'correct': 1526,
   'incorrect': 0,
   'partial': 1844,
   'missed': 1705,
   'spurious': 1590,
   'possible': 5075,
   'actual': 4960,
   'precision': 0.4935483870967742,
   'recall': 0.4823645320197044,
   'f1': 0.48789237668161434},
  'strict': {'correct': 1334,
   'incorrect': 2036,
   'partial': 0,
   'missed': 1705,
   'spurious': 1590,
   'possible': 5075,
   'actual': 4960,
   'precision': 0.26895161290322583,
   'recall': 0.26285714285714284,
   'f1': 0.26586945690084707},
  'exact': {'correct': 1526,
   'incorrect': 1844,
   'partial': 0,
   'missed': 1705,
   'spurious': 1590,
   'possible': 5075,
   'actual': 4960,
   'precision': 0.30766129032258066,
   'recall': 0.3006896551724138,
   'f1': 0.304135525660189

In [35]:
#results2['strict']
results_per_tag2

{'LOC': {'ent_type': {'correct': 108,
   'incorrect': 162,
   'partial': 0,
   'missed': 4805,
   'spurious': 36,
   'possible': 5075,
   'actual': 306,
   'precision': 0.35294117647058826,
   'recall': 0.0212807881773399,
   'f1': 0.04014123768816205},
  'partial': {'correct': 67,
   'incorrect': 0,
   'partial': 203,
   'missed': 4805,
   'spurious': 36,
   'possible': 5075,
   'actual': 306,
   'precision': 0.5506535947712419,
   'recall': 0.03320197044334976,
   'f1': 0.06262776435606765},
  'strict': {'correct': 29,
   'incorrect': 241,
   'partial': 0,
   'missed': 4805,
   'spurious': 36,
   'possible': 5075,
   'actual': 306,
   'precision': 0.09477124183006536,
   'recall': 0.005714285714285714,
   'f1': 0.010778665675524995},
  'exact': {'correct': 67,
   'incorrect': 203,
   'partial': 0,
   'missed': 4805,
   'spurious': 36,
   'possible': 5075,
   'actual': 306,
   'precision': 0.21895424836601307,
   'recall': 0.013201970443349753,
   'f1': 0.02490243449173016}},
 'ORG': 

In [32]:
#results3['strict']
results_per_tag3

{'LOC': {'ent_type': {'correct': 131,
   'incorrect': 145,
   'partial': 0,
   'missed': 52,
   'spurious': 122,
   'possible': 328,
   'actual': 398,
   'precision': 0.32914572864321606,
   'recall': 0.39939024390243905,
   'f1': 0.36088154269972456},
  'partial': {'correct': 168,
   'incorrect': 0,
   'partial': 108,
   'missed': 52,
   'spurious': 122,
   'possible': 328,
   'actual': 398,
   'precision': 0.5577889447236181,
   'recall': 0.676829268292683,
   'f1': 0.6115702479338843},
  'strict': {'correct': 82,
   'incorrect': 194,
   'partial': 0,
   'missed': 52,
   'spurious': 122,
   'possible': 328,
   'actual': 398,
   'precision': 0.20603015075376885,
   'recall': 0.25,
   'f1': 0.22589531680440772},
  'exact': {'correct': 168,
   'incorrect': 108,
   'partial': 0,
   'missed': 52,
   'spurious': 122,
   'possible': 328,
   'actual': 398,
   'precision': 0.4221105527638191,
   'recall': 0.5121951219512195,
   'f1': 0.46280991735537186}},
 'ORG': {'ent_type': {'correct': 277

In [76]:
def results(data):
    for tag, stats in data.items():
        print(tag + ':', 
              'precision:', str(round(stats['strict']['precision'], 4)) + ',', 
              'recall:', str(round(stats['strict']['recall'], 4)) + ',',
              'f1:', str(round(stats['strict']['f1'], 4)))

results(results_per_tag2)

LOC: precision: 0.08, recall: 0.0122, f1: 0.0212
ORG: precision: 0.7778, recall: 0.254, f1: 0.383
PER: precision: 0.5338, recall: 0.1471, f1: 0.2307
EVENT: precision: 0, recall: 0.0, f1: 0
DATE: precision: 0.0, recall: 0, f1: 0
MONEY: precision: 0.2213, recall: 0.0433, f1: 0.0724


In [46]:
def results2():
    tags1 = ['LOC', 'ORG', 'PER']
    tags2 = ['LOC', 'ORG', 'PER', 'EVENT', 'DATE', 'MONEY']
    tags3 = ['LOC', 'ORG', 'PER']

    result_estbertner = "Estbertner - "
    result_estbertner2 = "Estbertner2 - "
    result_bertner = "Bertner - "

    for tag in tags1:
        result_estbertner += tag + " - "
        result_estbertner += "precision: " + str(round(results_per_tag1[tag]['strict']['precision'], 4)) + ", "
        result_estbertner += "recall: " + str(round(results_per_tag1[tag]['strict']['recall'], 4)) + ", "
        result_estbertner += "f1: " + str(round(results_per_tag1[tag]['strict']['f1'], 4)) + " "

    for tag in tags2:
        result_estbertner2 += tag + " - "
        result_estbertner2 += "precision: " + str(round(results_per_tag2[tag]['strict']['precision'], 4)) + ", "
        result_estbertner2 += "recall: " + str(round(results_per_tag2[tag]['strict']['recall'], 4)) + ", "
        result_estbertner2 += "f1: " + str(round(results_per_tag2[tag]['strict']['f1'], 4)) + " "

    for tag in tags3:
        result_bertner += tag + " - "
        result_bertner += "precision: " + str(round(results_per_tag3[tag]['strict']['precision'], 4)) + ", "
        result_bertner += "recall: " + str(round(results_per_tag3[tag]['strict']['recall'], 4)) + ", "
        result_bertner += "f1: " + str(round(results_per_tag3[tag]['strict']['f1'], 4)) + " "

    print(result_estbertner)
    print(result_estbertner2)
    print(result_bertner)

results()

Estbertner - LOC - precision: 0.4064, recall: 0.4648, f1: 0.4337 ORG - precision: 0.6484, recall: 0.3796, f1: 0.4789 PER - precision: 0.699, recall: 0.8175, f1: 0.7536 
Estbertner2 - LOC - precision: 0.08, recall: 0.0122, f1: 0.0212 ORG - precision: 0.7778, recall: 0.254, f1: 0.383 PER - precision: 0.5338, recall: 0.1471, f1: 0.2307 EVENT - precision: 0, recall: 0.0, f1: 0 DATE - precision: 0.0, recall: 0, f1: 0 MONEY - precision: 0.0, recall: 0.0, f1: 0 
Bertner - LOC - precision: 0.206, recall: 0.25, f1: 0.2259 ORG - precision: 0.3092, recall: 0.1193, f1: 0.1722 PER - precision: 0.6353, recall: 0.7788, f1: 0.6998 


In [18]:
round(results_per_tag3['LOC']['strict']['f1'], 4)

0.1585

In [197]:
temp_x = []
for i in x['1922-04-24_manual_annotated.json']:
    if i[2] == 'O':
        temp_x.append(i[1])
    else:
        y = i[1].split()
        for i in y:
            temp_x.append(i)
len(temp_x)

2225

In [77]:
with open(os.path.join(OutputFolder, '1922-04-24_manual_annotated.json'), "r", encoding="utf-8") as file:
    content = file.read()
    
text_import = json_to_text(json_text=content)

temp = 0
for i in text_import.ne_gold_a:
    temp += len(i.text.split()) - 1

print(temp)
print(len(x['1922-04-24_manual_annotated.json']))
print(len(text_import.words))

164
2105
2225


In [29]:
temp = 0
for i in text_import.words:
    temp += len(i.text)
temp

11419

In [87]:
temp = 0
for i in x['1922-04-24_manual_annotated.json']:
    #temp += sum([len(j) for j in i[1].split()])
    temp += sum([len(j) for j in i[1].split(" ")])
temp

11419

In [20]:
temp = 0
for i in text_import.sentences:
    temp += sum([len(j) for j in i.text])
temp

11419

In [51]:
conflict(text_import.sentences[0], text_import.ne_gold_a[0])

True

In [73]:
text_import.words[17:26]

Layer(name='words', attributes=('normalized_form',), spans=SL[Span('Juhatab', [{'normalized_form': None}]),
Span('linnawolikogu', [{'normalized_form': None}]),
Span('esimees', [{'normalized_form': None}]),
Span('P.Pöld', [{'normalized_form': None}]),
Span('Koos', [{'normalized_form': None}]),
Span('on', [{'normalized_form': None}]),
Span('40', [{'normalized_form': None}]),
Span('linnawolinikku', [{'normalized_form': None}]),
Span('.', [{'normalized_form': None}])])

In [75]:
print(text_import.sentences[0].start, text_import.sentences[0].end)
print(text_import.words[:17][0].start, text_import.words[:17][-1].end)
print(text_import.sentences[1].start, text_import.sentences[1].end)
print(text_import.words[17:26][0].start, text_import.words[17:26][-1].end)

0 92
0 92
93 156
93 156


In [ ]:
def next_tag(text_import, tagnr):
    """
    Find the next tag with a valid type.
    
    Args:
        text_import: Text object with NE annotations
        tagnr: Current tag index
    
    Returns:
        Index of next valid tag or -1 if none found
    """
    tags = {
        "PER": "PER",
        "LOC": "LOC",
        "ORG": "ORG",
        "ORG_POL": "ORG",
        "ORG_GPE": "ORG",
    }
    
    tagnr += 1
    
    while True:
        if tagnr >= len(text_import.ne_gold_a):
            return -1
        if text_import.ne_gold_a[tagnr].tag in tags:
            return tagnr
        else:
            tagnr += 1

def process_file(file_path):
    """
    Process a file and convert to NER format.
    
    Args:
        file_path: Path to the file to process
        
    Returns:
        List of [sentence_id, text, tag] entries
    """
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()
    
    text_import = json_to_text(json_text=content)
    
    data = []
    tag_nr = next_tag(text_import, -1)
    sen_nr = 0
    
    tags = {
        "PER": "PER",
        "LOC": "LOC",
        "ORG": "ORG",
        "ORG_POL": "ORG",
        "ORG_GPE": "ORG",
    }

    # Process each word
    for word in text_import.words:
        # Update sentence ID if needed
        #print(conflict(text_import.sentences[sen_nr], word), text_import.sentences[sen_nr].text, word.text)
        if sen_nr < len(text_import.sentences) and not conflict(text_import.sentences[sen_nr], word):
            sen_nr += 1
        #print(sen_nr, word, text_import.ne_gold_a[tag_nr], conflict(text_import.ne_gold_a[tag_nr], word))
        # Check if this word is part of a named entity
        if tag_nr != -1:
            tag = text_import.ne_gold_a[tag_nr]
            
            # If word is before the current entity
            if word.end < tag.start:
                # Add as regular word
                data.append([sen_nr, word.text, "O"])
            
            # If word is part of the current entity
            elif conflict(tag, word):
                # Skip this word - it's part of an entity that will be added when we reach its end
                continue
                
            # If word is after the current entity
            elif word.start >= tag.end:
                # We've passed the current entity, add it and move to next
                #print("yes", word, sen_nr, tag.end, word.start)
                normalized_tag = tags.get(tag.tag, tag.tag)
                data.append([sen_nr, tag.text, f"I-{normalized_tag}"])
                tag_nr = next_tag(text_import, tag_nr)
                
                # Now check the current word again
                if tag_nr != -1:
                    tag = text_import.ne_gold_a[tag_nr]
                    
                    # If word is part of the next entity
                    if conflict(tag, word):
                        continue

                    else:
                        data.append([sen_nr, word.text, "O"])
                else:
                    # No more entities, add as regular word
                    data.append([sen_nr, word.text, "O"])
        else:
            # No entities or no more entities, add as regular word
            data.append([sen_nr, word.text, "O"])
    
    # Make sure to add the last entity if we haven't reached it yet
    if tag_nr != -1:
        print("yes")
        normalized_tag = tags.get(text_import.ne_gold_a[tag_nr].tag, text_import.ne_gold_a[tag_nr].tag)
        data.append([sen_nr, text_import.ne_gold_a[tag_nr].text, f"I-{normalized_tag}"])
    
    return data

def process_directory_to_dict(output_folder):
    results_dict = {}
    
    for name in os.listdir(output_folder):
        file_path = os.path.join(output_folder, name)
        results_dict[name] = process_file(file_path)
        break
    
    return results_dict

x = process_directory_to_dict('output2')
x['1922-04-24_manual_annotated.json']

In [93]:
for nr in range(len(x['1922-04-24_manual_annotated.json'][:-1])):
    if x['1922-04-24_manual_annotated.json'][nr][2] != 'O' and x['1922-04-24_manual_annotated.json'][nr][2] == x['1922-04-24_manual_annotated.json'][nr+1][2]:
        print(x['1922-04-24_manual_annotated.json'][nr], x['1922-04-24_manual_annotated.json'][nr+1])

[55, 'linnawalitsus', 'I-ORG'] [55, 'wolikogule', 'I-ORG']
[65, 'linnawalitsus', 'I-ORG'] [65, 'wolikogule', 'I-ORG']
[81, 'linnawalitsus', 'I-ORG'] [81, 'wolikogule', 'I-ORG']
[106, 'Tartu ülikooliwalitsus', 'I-ORG'] [106, 'linnawalitsuse', 'I-ORG']
[128, 'Linnawalitsuse', 'I-ORG'] [128, 'Ameerika Punase Risti', 'I-ORG']


In [46]:
def next_tag(text_import, tagnr):
    tags = {
        "PER": "PER",
        "LOC": "LOC",
        "ORG": "ORG",
        "ORG_POL": "ORG",
        "ORG_GPE": "ORG",
    }
    
    tagnr += 1
    
    while True:
        if tagnr >= len(text_import.ne_gold_a):
            return -1
        if text_import.ne_gold_a[tagnr].tag in tags:
            return tagnr
        else:
            tagnr += 1

#def process_file(file_path):
def process_file(file_name):
    #with open(file_path, "r", encoding="utf-8") as file:
    with open(os.path.join(OutputFolder, file_name), "r", encoding="utf-8") as file:
        content = file.read()
    
    text_import = json_to_text(json_text=content)
    
    data = []
    tag_nr = next_tag(text_import, -1)
    sen_nr = 0
    
    tags = {
        "PER": "PER",
        "LOC": "LOC",
        "ORG": "ORG",
        "ORG_POL": "ORG",
        "ORG_GPE": "ORG",
    }

    # Process each word
    for word in text_import.words:
        # Update sentence ID if needed
        if sen_nr < len(text_import.sentences) and not conflict(text_import.sentences[sen_nr], word):
            sen_nr += 1
        if file_name == '1932-01-25_manual_annotated.json' and sen_nr == 146:
            print(word, text_import.ne_gold_a[tag_nr])
        # Check if this word is part of a named entity
        if tag_nr != -1:
            tag = text_import.ne_gold_a[tag_nr]
            
            # If word is before the current entity
            if word.end <= tag.start:
                # Add as regular word
                data.append([file_name, sen_nr, word.text, "O"])

            if word.start == tag.start:
                normalized_tag = tags.get(tag.tag, tag.tag)
                data.append([file_name, sen_nr, word.text, f"B-{normalized_tag}"])
            # If word is part of the current entity
            elif conflict(tag, word):
                if file_name == '1932-01-25_manual_annotated.json' and sen_nr == 146:
                    print('yes')
                data.append([file_name, sen_nr, word.text, f"I-{normalized_tag}"])
                
            # If word is after the current entity
            elif word.start >= tag.end:
                tag_nr = next_tag(text_import, tag_nr)
                
                if tag_nr != -1:
                    tag = text_import.ne_gold_a[tag_nr]

                    if word.start == tag.start:
                        data.append([file_name, sen_nr, word.text, f"B-{normalized_tag}"])
                    else:
                        data.append([file_name, sen_nr, word.text, "O"])
                else:
                    data.append([file_name, sen_nr, word.text, "O"])
        else:
            data.append([file_name, sen_nr, word.text, "O"])
    
    return data

def process_directory_to_dict(output_folder):
    all_data = []
    
    for name in os.listdir(output_folder):
        all_data.extend(process_file(name))

    data = pd.DataFrame(all_data, columns=["filename", "sentence_id", "words", "labels"])
    return data

data = process_directory_to_dict('output2')
data

Span('Ln.J.Sütt', [{'normalized_form': None}]) Span('J.Sütt', [{'tag': 'PER'}])
yes
Span(':', [{'normalized_form': None}]) Span('J.Sütt', [{'tag': 'PER'}])
Span('Iseäralikud', [{'normalized_form': None}]) Span("Lill'il", [{'tag': 'PER'}])
Span('motiivid', [{'normalized_form': None}]) Span("Lill'il", [{'tag': 'PER'}])
Span('on', [{'normalized_form': None}]) Span("Lill'il", [{'tag': 'PER'}])
Span('härra', [{'normalized_form': None}]) Span("Lill'il", [{'tag': 'PER'}])
Span("Lill'il", [{'normalized_form': None}]) Span("Lill'il", [{'tag': 'PER'}])
Span(',', [{'normalized_form': None}]) Span("Lill'il", [{'tag': 'PER'}])
Span('tuues', [{'normalized_form': None}]) Span('Lillil', [{'tag': 'PER'}])
Span('turu', [{'normalized_form': None}]) Span('Lillil', [{'tag': 'PER'}])
Span('kohta', [{'normalized_form': None}]) Span('Lillil', [{'tag': 'PER'}])
Span('andmeid', [{'normalized_form': None}]) Span('Lillil', [{'tag': 'PER'}])
Span(',', [{'normalized_form': None}]) Span('Lillil', [{'tag': 'PER'}])
S

,filename,sentence_id,words,labels
0,1922-04-24_manual_annotated.json,0,Tartu,B-ORG
1,1922-04-24_manual_annotated.json,0,linnawolikogu,I-ORG
2,1922-04-24_manual_annotated.json,0,korraline,O
3,1922-04-24_manual_annotated.json,0,koosolek,O
4,1922-04-24_manual_annotated.json,0,esmaspäewal,O
...,...,...,...,...
51241,1941-01-03_manual_annotated.json,42,.,O
51242,1941-01-03_manual_annotated.json,43,Ärakiri,O
51243,1941-01-03_manual_annotated.json,43,õige,O
51244,1941-01-03_manual_annotated.json,43,:,O


In [183]:
len(x['1922-04-24_manual_annotated.json'])

2225

In [111]:
len(text_import.words)

2225

In [185]:
for i in range(len(x['1922-04-24_manual_annotated.json'])):
    if x['1922-04-24_manual_annotated.json'][i][1] != text_import.words[i].text:
        print(i, x['1922-04-24_manual_annotated.json'][i][0],  x['1922-04-24_manual_annotated.json'][i][1], text_import.words[i].text)

In [193]:
train_data = pd.DataFrame(
    x['1922-04-24_manual_annotated.json'], columns=["sentence_id", "words", "labels"]
)

In [189]:
print(list(train_data[train_data['sentence_id'] == 62]['words']))
for i in range(len(list(train_data[train_data['sentence_id'] == 62]['words']))):
    if list(train_data[train_data['sentence_id'] == 62]['words'])[i] != text_import.sentences[62].text[i]:
        print(list(train_data[train_data['sentence_id'] == 62]['words'])[i], text_import.sentences[62].text[i])

['Kurs-Oleski', 'ettepanek', '-', '6', 'poolt', ',', '28', 'wastu', ',', '1', 'erapooletu', 'Linnawalitsuse', 'ettepanek', '-11', '"', '24', '"', '-', 'Lwol', '.', 'J.Kala', 'ettepanek', '-', '10', '"', '22', '"', '2', '"', 'Lwol', '.', 'O.Alliku', '"', '-', '7', '"', '26', '"', '2', '"', 'Lwol', '.', 'K.Leppa', '"', '-', '16', '"', '22', '"', '-', 'Seega', 'on', 'küsimus', 'lahtiseks', 'jäänud', ',', 'sest', 'et', 'mitte', 'ükski', 'ettepanek', 'tarwilikku', 'häälte', 'enamust', 'ei', 'saanud', '.']


In [167]:
train_data.loc[690:720]

,sentence_id,words,labels
690,62,"""",O
691,62,24,O
692,62,"""",O
693,62,-,O
694,62,Lwol,O
695,62,.,O
696,62,J.Kala,B-PER
697,62,ettepanek,O
698,62,-,O
699,62,10,O


In [25]:
#Testing if every word is present in the dataframe
for name in os.listdir(OutputFolder):
    with open(os.path.join(OutputFolder, name), "r", encoding="utf-8") as file:
        content = file.read()
    text_import = json_to_text(json_text=content)

    print(len(data[data["filename"] == name]) == len(text_import.words), len(data[data["filename"] == name]), len(text_import.words))

True 2225 2225
True 7245 7245
True 12793 12793
True 8978 8978
True 11731 11731
True 7583 7583
True 691 691


In [21]:

for name in os.listdir(OutputFolder):
    with open(os.path.join(OutputFolder, name), "r", encoding="utf-8") as file:
        content = file.read()
    text_import = json_to_text(json_text=content)

    tags = {
        "PER": "PER",
        "LOC": "LOC",
        "ORG": "ORG",
        "ORG_POL": "ORG",
        "ORG_GPE": "ORG",
    }
    
    starting_data = [[i.text, tags[i.tag]] for i in text_import.ne_gold_a if i.tag in list(tags.keys())]
    print(
        len(data[(data["filename"] == name) & (data["labels"].str.startswith('B-'))]) == len(starting_data),
        len(data[(data["filename"] == name) & (data["labels"].str.startswith('B-'))]),
        len(starting_data),
        name
    )
    print('----------')

    for nr, (idx, row) in enumerate(data[(data["filename"] == name) & (data["labels"].str.startswith('B-'))].iterrows()):
        if starting_data[nr][0].startswith(row['words']) == False:
            print(nr, starting_data[nr], row)
        #print(i)
    #break

    #print(starting_data[-10:])
    #print(data[(data["filename"] == name) & (data["labels"].str.startswith('B-'))].tail(10)['words']) 
    
    #print(text_import.ne_gold_a.tag)
    #print(data[(data["filename"] == name) & (data["labels"] != 'O')][:20])
    
    #print(starting_data)

True 166 166 1922-04-24_manual_annotated.json
----------
True 495 495 1927-03-28_manual_annotated.json
----------
False 720 723 1932-01-25_manual_annotated.json
----------
153 ['J.Sütt', 'PER'] filename       1932-01-25_manual_annotated.json
sentence_id                                 146
words                                   Lill'il
labels                                    B-PER
Name: 11678, dtype: object
154 ["Lill'il", 'PER'] filename       1932-01-25_manual_annotated.json
sentence_id                                 147
words                                    Lillil
labels                                    B-PER
Name: 11705, dtype: object
155 ['Lillil', 'PER'] filename       1932-01-25_manual_annotated.json
sentence_id                                 153
words                           Linnavalitsusel
labels                                    B-ORG
Name: 11780, dtype: object
156 ['Linnavalitsusel', 'ORG'] filename       1932-01-25_manual_annotated.json
sentence_id              

In [ ]:
name = '1932-01-25_manual_annotated.json'
with open(os.path.join(OutputFolder, name), "r", encoding="utf-8") as file:
        content = file.read()
text_import = json_to_text(json_text=content)
#text_import.sentences[146].display()
#text_import.ne_gold_a[154]
text_import.ne_gold_a.display()